In [1]:
from rubiks import *
from solver import *
from heuristics import *
import timeit
import pandas as pd
import pickle

In [2]:
h = {
    "numberOfCorrectColorsOnFace": numberOfCorrectColorsOnFace,
    "numberOfPairedCornerNEdges": numberOfPairedCornerNEdges,
    "oneStepCornerEdgePair": oneStepCornerEdgePair,
    "compound1": compound1,
    "compound2": compound2,
    "numberOfSolidRows": numberOfSolidRows
}

In [3]:
results = {}

In [4]:
# unload scrambles
filename = 'testScrambles.pkl'

with open(filename, "rb") as fp:   # Unpickling
    scrambles = pickle.load(fp)

In [5]:
all_steps = [3,5,7,10]
# all_steps = [3,5]
eps = 20
scrambles = {steps: {} for steps in all_steps}
# make scrambles
for steps in all_steps:
    for i in range(eps):
        scramble = generateScramble(steps)
        scrambles[steps][scramble] = None
    

In [6]:
results  = {name: {} for name in h}
for (name, heuristic) in h.items():
    result = {}
    for step, step_scrambles in scrambles.items():
        time = 0
        solved = 0
        for scramble in step_scrambles:
            c = cube()
            c.move(scramble)

            s = solver(c, h=heuristic)
            start = timeit.default_timer()
            s.solve(maxDepth=10, display=False, verbose=False, earlyTermination=500)
            if s.solutionFound:
                solved +=1
            stop = timeit.default_timer()
            timeTaken = stop - start
            time += timeTaken

        avg_time = time/len(step_scrambles)
        result['{}_steps_avg_time'.format(step)] = avg_time
        result['{}_steps_solves (%)'.format(step)] = (solved/len(step_scrambles)) * 100
        results[name] = result
        

In [7]:
pd.DataFrame.from_dict(results, orient='index')

,3_steps_avg_time,3_steps_solves (%),5_steps_avg_time,5_steps_solves (%),7_steps_avg_time,7_steps_solves (%),10_steps_avg_time,10_steps_solves (%)
numberOfCorrectColorsOnFace,0.002504,95.0,0.056990,50.0,0.124904,20.0,0.160886,5.0
numberOfPairedCornerNEdges,0.003148,80.0,0.038421,75.0,0.416739,45.0,0.327570,0.0
oneStepCornerEdgePair,1.988007,55.0,7.347173,25.0,5.357121,20.0,5.456237,0.0
compound1,0.007283,75.0,0.076787,55.0,0.310608,40.0,1.862958,5.0
compound2,0.004848,75.0,0.115054,65.0,0.390590,35.0,0.528538,10.0
numberOfSolidRows,0.003479,100.0,0.014480,80.0,0.137706,55.0,0.440204,15.0
